In [20]:
from alphatoe import models, plot, interpretability, game
import pandas as pd
import torch
from pytorch_memlab import LineProfiler, MemReporter
from showmethetypes import SMTT
import einops

In [2]:
tt = SMTT("torch")

In [3]:
model = interpretability.load_model(
    "../scripts/models/prob all 8 layer control-20230718-185339"
)

In [4]:
games = pd.read_csv("../data/prob all 8 layer control-20230718-185339_stats.csv")
games.head()

/tmp/ipykernel_47258/629368561.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv("../data/prob all 8 layer control-20230718-185339_stats.csv")


,moves played,steps till end state,winner,rotation 1,rotation 2,rotation 3,horizontal flip,vertical flip,training index,train or test,first win condition,second win condition,end move loss
0,"[0, 1, 3, 2, 6]",5,X,399,1439,1040,1114,325,241912,test,left column,NaN,0.000005
1,"[0, 1, 3, 4, 6]",5,X,396,1438,1043,1112,327,190522,train,left column,NaN,0.000114
2,"[0, 1, 3, 5, 6]",5,X,398,1437,1041,1113,326,90275,train,left column,NaN,0.000007
3,"[0, 1, 3, 7, 6]",5,X,395,1436,1044,1110,329,21994,train,left column,NaN,0.000008
4,"[0, 1, 3, 8, 6]",5,X,397,1435,1042,1111,328,48696,train,left column,NaN,0.000013


- Sort games by game length [X]
- batch inference for games of length [X]
- extract activations with hooks [X]
- train autoencoder on data reconstruction (anthropic has tips here)
- find good metrics + start looking at data (anthropic has tips here)

In [5]:
all_games = game.generate_all_games([game.Board()])

In [6]:
len(all_games)

255168

In [7]:
games_len_dict = {5: [], 6: [], 7: [], 8: [], 9: []}
for game in all_games:
    games_len_dict[len(game.moves_played)].append(game.moves_played)

In [8]:
s = 0
for key in games_len_dict.keys():
    s += len(games_len_dict[key])
print(s)

255168


In [9]:
games_len_tensors = {}
for key in games_len_dict.keys():
    games_len_tensors[key] = torch.stack(
        [torch.tensor([10] + game) for game in games_len_dict[key]]
    )

In [10]:
games_len_tensors[9]

tensor([[10,  0,  1,  ...,  7,  6,  8],
        [10,  0,  1,  ...,  7,  8,  6],
        [10,  0,  1,  ...,  5,  7,  8],
        ...,
        [10,  8,  7,  ...,  3,  1,  0],
        [10,  8,  7,  ...,  1,  0,  2],
        [10,  8,  7,  ...,  1,  2,  0]])

In [11]:
# Activation hook
def neuron_activations(seq):
    def hook(module, input, output):
        result = output.clone()
        module.captured_activations = result

    try:
        with torch.inference_mode():
            handle = model.blocks[0].mlp.hook_post.register_forward_hook(hook)
            _ = model(seq)
            activations = model.blocks[0].mlp.hook_post.captured_activations
            handle.remove()
    except Exception as e:
        handle.remove()
        raise e

    return activations

In [12]:
# a = neuron_activations(games_len_tensors[5])[:, -1]

In [13]:
# a.numel() * a.element_size()

In [18]:
b = neuron_activations(games_len_tensors[9])
reporter = MemReporter()
reporter.report()

Element type                                            Size  Used MEM
-------------------------------------------------------------------------------
Storage on cuda:0
Tensor0                                       (1440, 6, 512)    16.88M
Tensor1                                       (5328, 7, 512)    72.84M
Tensor2                                      (47952, 8, 512)   749.25M
Tensor3                                      (72576, 9, 512)     1.25G
Tensor4                                    (127872, 10, 512)     2.44G
Tensor5                                    (127872, 10, 512)     2.44G
Parameter6                                         (11, 128)     5.50K
Parameter7                                         (10, 128)     5.00K
Parameter8                                         (128, 10)     5.00K
Parameter9                                             (10,)   512.00B
Parameter10                                     (8, 128, 16)    64.00K
Parameter11                                     (8

In [15]:
torch.cuda.empty_cache()

In [16]:
assert False

AssertionError: 

In [ ]:
b.numel() * b.element_size()

261881856

In [ ]:
for elt in all_acts:
    print(elt.numel() * elt.element_size())

NameError: name 'all_acts' is not defined

In [17]:
# Inference loop
# will there be a difference across game lengths?
# Doesn't fit in memory?
# 512 neurons * 255168 games * 32 bit floats * 10 seq len = 5.22 gigabytes???
all_acts = []
for i, key in enumerate(games_len_tensors.keys()):
    print(i)
    acts = neuron_activations(games_len_tensors[key])
    all_acts.append(acts)
    torch.cuda.empty_cache()

0
1
2
3
4


In [ ]:
tt(all_acts)

list (5)
    |__Tensor (dtype: torch.float32)
        |  (device: cuda:0)
        |__dim_0 (1440)
        |__dim_1 (512)


In [21]:
act_data = torch.cat(
    [einops.rearrange(acts, "batch seq dim -> (batch seq) dim") for act in all_acts],
    dim=0,
)

Note that since there's a lot of repeated phrases in the input, we'll have lots of identical activations. Not sure how that'll change things yet though..

In [22]:
tt(act_data)

Tensor (dtype: torch.float32)
    |  (device: cuda:0)
    |__dim_0 (6393600)
    |__dim_1 (512)


In [ ]:
autoenc = models.SparseAutoEncoder(512, 1024).to("cuda")

In [ ]:
loss_fn = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(autoenc.parameters(), lr=1e-4, weight_decay=0)

In [ ]:
epochs = 10
batch_size = 2**15
lam = 1e-2
for epoch in range(epochs):
    for batch in range(0, act_data.shape[0], batch_size):
        reg, guess = autoenc(act_data[batch : batch + batch_size])
        loss = loss_fn(guess, act_data[batch : batch + batch_size]) + lam * reg
        losses.append(interpretability.numpy(loss))
        optimizer.zero_grad()
        loss.backward()
        print(loss)
        optimizer.step()

In [25]:
2**14

16384